In [3]:
import os.path
import pickle
from pathlib import Path
import autofit as af
from warm_pixels import hst_utilities as ut


data_directory = Path(os.path.abspath("")) / "data"


def load_data(name):
    with open(data_directory / f"{name}.pickle", "r+b") as f:
        return pickle.load(f)


date = load_data("date")


# CCD
beta = 0.478
w = 84700.0
# Trap species
A = 0.17
B = 0.45
C = 0.38
# Trap lifetimes before or after the temperature change
if date < ut.date_T_change:
    tau_a = 0.48
    tau_b = 4.86
    tau_c = 20.6
else:
    tau_a = 0.74
    tau_b = 7.70
    tau_c = 37.0


In [ ]:

class Analysis(af.Analysis):
    def __init__(self, x_all, y_all, noise_all, n_e_all, n_bg_all, row_all):
        self.x_all = x_all
        self.y_all = y_all
        self.noise_all = noise_all
        self.n_e_all = n_e_all
        self.n_bg_all = n_bg_all
        self.row_all = row_all

    def log_likelihood_function(self, instance):
        pass